# The Three Prisoners Problem

Three prisoners, Alice, Bob and Charlie are senteced to death, but one of them (uniformly chosen at random) is selected to be pardoned, so that just the two out of the three prisoners will be executed. The warden knows which one will be pardoned, but he is not allowed to tell the prisoners. Alice begs the warden to let her know the identity of one of the others who will be executed saying: 

> _"If Bob is pardoned, say Charlie's name, and if Charlie is pardoned say Bob's. If I'm pardoned, chose randomly to name Bob or Charlie."_

Now, we are interested in answering the following two questions:
1. Given the warden's answer, what is the probability of correctly guessing the pardoned prisoner?
2. Is the warden’s answer useful for Alice?

## Channel matrix 

Let's model this problem using a channel $W$ that takes an input $X$ (the prisoner to be pardoned) and produces an output $Y$ (the warden's answer). You can think of $W$ as being the warden in our problem. The possible values for $X$ and $Y$ are $A, B, C$ (short for Alice, Bob and Charlie). Notice that the warden will never say Alice's name, but still for the more general case we include it. $W$ is defined as

$$
W = \left( \begin{array} {ccc}
    p(Y=A | X=A) & p(Y=B | X=A) & p(Y=C | X=A) \\
    p(Y=A | X=B) & p(Y=B | X=B) & p(Y=C | X=B) \\
    p(Y=A | X=C) & p(Y=B | X=C) & p(Y=C | X=C) \\
\end{array} \right) 
$$


or in our case

$$ 
W = \left( \begin{array} {cc}
    0 & \frac{1}{2} & \frac{1}{2} \\
    0 & 0 & 1 \\
    0 & 1 & 0 \\
\end{array} \right)
$$

$W$'s first row corresponds to $X=A$, meaning the scenario where Alice is chosen to be pardoned. In that case the channel's output (or the warden's saying) is not deterministic, but has some degree of randomness. More specifically the warden says Alice with probability $0$, and chooses uniformly between Bob and Charlie, that is, with probability $\frac{1}{2}$ each. 

$W$'s second row corresponds to $X=B$, meaning the scenario where Bob is chosen to be pardoned. In that case, there is only one possible output, and that is Charlie. In this case, $W$ (the warden) behaves deterministically and this can be seen because the second row of $W$ has $0$ everywhere, except for one specific output, which gets probability $1$. Same happens with the third row, which corresponds to $X=C$, meaning Charlie is chosen to be pardoned.

Notice that each row of $W$ sums up to $1$. This happens because each row defines a probability distribution, which basically says how $W$ (the warden) behaves given a specific input $X$.

Let's also define $W$ using python and libqif:

In [2]:
import numpy as np
import matplotlib.pyplot as plt
try:
    from qif import *
except: # install qif if not available (for running in colab, etc)
    import IPython; IPython.get_ipython().run_line_magic('pip', 'install qif')
    from qif import *

In [3]:
W = np.array([
    # Y=A  Y=B  Y=C
    [   0, 1/2, 1/2],    # X=A
    [   0,   0,   1],    # X=B
    [   0,   1,   0],    # X=C
])

Now, to answer Question 1 using QIF terminology, we want to find the **posterior vulnerability** of $W$. That is, what is the probability of correctly guessing the secret $X$ after observing the channel's output $Y$. Let's see how we can compute that.

## Prior distribution 

First of all we must define the distribution of $X$. It is also called *the prior distribution* $\pi$. In our case that is the probability of each prisoner being pardoned. The problem states that it is uniform, so we have

$$
p(X=A) = p(X=B) = p(X=C) = \frac{1}{3} 
$$

or for short

$$
\pi = (\frac{1}{3},\frac{1}{3}, \frac{1}{3}) \\
$$

Let's also define that in python:

In [4]:
pi = probab.uniform(3)

## Joint Matrix

Next, we compute $J$, which is defined as

$$ 
J = \left( \begin{array} {ccc}
    p(Y=A \textit{ and } X=A) & p(Y=B \textit{ and } X=A) & p(Y=C \textit{ and } X=A) \\
    p(Y=A \textit{ and } X=B) & p(Y=B \textit{ and } X=B) & p(Y=C \textit{ and } X=B) \\
    p(Y=A \textit{ and } X=C) & p(Y=B \textit{ and } X=C) & p(Y=C \textit{ and } X=C) \\
\end{array} \right)
$$

$J$ contains the joint probabilities for each combination of $X$ and $Y$. For computing $J$, we use the rule $p_{Y,X}(y,x) = p_X(x) \cdot p_{Y|X}(y|x) $.

$$ 
J = \left( \begin{array} {ccc}
    p(X=A) \cdot p(Y=A | X=A) & p(X=A) \cdot p(Y=B | X=A) & p(X=A) \cdot p(Y=C | X=A) \\ 
    p(X=B) \cdot p(Y=A | X=B) & p(X=B) \cdot p(Y=B | X=B) & p(X=B) \cdot p(Y=C | X=B) \\
    p(X=C) \cdot p(Y=A | X=C) & p(X=C) \cdot p(Y=B | X=C) & p(X=C) \cdot p(Y=C | X=C) \\
\end{array} \right)
$$

Notice that $J$ depends on the channel $W$, **but also** on the distribution $\pi$ of $X$. Meaning that it depends on all of the $p(X=A)$, $p(X=B)$, $p(X=C)$. Thus, if the pardoned prisoner were not chosen at random (meaning $\pi$ was different), then $J$ would be different.

If we compute $J$ for our case, it becomes

$$ 
J = \left( \begin{array} {cc}
    0 & \frac{1}{6} & \frac{1}{6} \\
    0 & 0 & \frac{1}{3} \\
    0 & \frac{1}{3} & 0 \\
\end{array} \right)
$$

See that if we sum all of $J$'s elements they add up to 1. That is excpected because by the defitition of probabiliy, when we sum the probabilities of every possible outcume, they must add up to 1. And that is exactly what happens when we sum $J$'s elements.

## Posterior Distributions
But we're most interested in *what exactly* does the warden's saying ($W$'s output) tells us about $X$.

To answer this, we define $P$ as

$$ 
P = \left( \begin{array} {ccc}
    p(X=A | Y=A) & p(X=A | Y=B) & p(X=A | Y=C) \\
    p(X=B | Y=A) & p(X=B | Y=B) & p(X=B | Y=C) \\
    p(X=C | Y=A) & p(X=C | Y=B) & p(X=C | Y=C) \\
\end{array} \right) 
$$

Take a moment to compare it with $W$. It seems similar. Except that the $X$'s and the $Y$'s have "switched places" over the vertical line.

$P$'s first column gives us the probabilities of each prisoner being pardoned, **given that** the warden has said Alice's name. That is, the probability of $X$ being $A, B$ or $C$, **given that** $W$'s output is $A$. The same happens with the second and the third column.

Notice that each column gives us a new probability distribution (also meaning that each column adds up to 1). That is, it tells us the **updated** probability  of each $X$ **after** observing $W$'s output.

$P$ can be computed using the rules $p(y) = \sum_{x \in X} p_{X, Y}(x, y)$ and $p_{X|Y}(x|y) = \frac{p_{X, Y}(x, y)}{p_Y(y)}$.

In our case, it becomes

$$ 
P = \left( \begin{array} {cc}
    0 & \frac{1}{3} & \frac{1}{3} \\
    0 & 0 & \frac{2}{3} \\
    0 & \frac{2}{3} & 0 \\
\end{array} \right)
$$

## Finally solving the problem

### Question 2

We are now ready to answer the two questions we set for ourselves. First we will answer the question number 2, which asks

>Is the warden’s answer useful for Alice?

Before hearing the warden's saying ($W$'s output), Alice knew that she had a $\frac{1}{3}$ probability of surviving (becuase the pardoned prisoner were chosen uniformly at random).

Now, using matrix $P$, we know that if the warden says Bob's name ($P$'s second column) then Alice has a $p(X=A | Y=B) = \frac{1}{3}$ ($P$'s second column and $P$'s first row) probability of surviving. 

With the same reasoning, we see that if the warden says Charlie's name ($P$'s third column) then Alice has a $p(X=A | Y=C) = \frac{1}{3}$ ($P$'s third column and $P$'s first row) probability of surviving.

Notice that the warden never says Alice's name ($W$ never outputs $A$) and this can be seen by verifying that $p_Y(A) = 0$.

So before the warden's answer, Alice survived with a probability of $\frac{1}{3}$. After the warden's answer, no matter which name the warden says, Alice **again** survives with a probabiliy of $\frac{1}{3}$.

Looks like Alice should have picked her question more carefully, because *the warden's answer is never useful for Alice*.

### Question 1

Now we will answer question 1, which asks

>Given the warden's answer, what is the probability of correctly guessing the pardoned prisoner?

Consider the case that the warden says Bob's name ($P$'s second column). Alice has a probability of $\frac{1}{3}$ of being pardoned, Bob $0$ and Charlie $\frac{2}{3}$. What would be your best guess for who has been pardoned? The logical answer is Charlie, because he has the highest probability among the others **in that specific column**. So if the warden says Bob's name, you know that your best guess is Charlie and you have a probability of success $\frac{2}{3}$.

Using the same reasoning, we can see that if the warden says Charlie's name ($P$'s third column), your best guess would Bob because he has the highest probability **in that specific column**. So again, if the warden says Charlie's name, you know that your best guess is Bob and you have a probability of success $\frac{2}{3}$.

So before the warden's answer, our best guess for who is the pardoned prisoner would be, well, anyone since they have the same probability of being pardoned. And our probability of success is $\frac{1}{3}$. 

>This is also known as *prior vulnerability* of $W$. That is, the probability of correctly guessing the secret $W$ *before* observing the channel's output $Y$.

But after receiving the warden's answer ($W$'s output) we see that we can make a guess with a probability of success $\frac{2}{3}$! In the first case we have to guess Charlie and in the second Bob. 

So to answer the question, *given the warden's answer, the probability of correctly guessing the pardoned prisoner is* $\frac{2}{3}$.

>This is also known as the *posterior vulnerability* of $W$, also named $V(\pi, C)$. That is, the probability of correctly guessing the secret $W$ *after* observing the channel's output $Y$.

Alice's name never pops up so we don't have to deal with that. In fact we could use the same reasoning if we eliminated $P$'s first row, since it has $0$ everywhere.

Generalizing this, we can compute the posterior vulnerability of a channel $W$ by first computing its posterior distribution matrix $P$ then finding for each column the maximum element (which corresponds to the best guess when $W$'s output is the one corresponding to that column). In our case the maximum elements were $\frac{2}{3}$ and $\frac{2}{3}$. 

But we need a way to combine them and get one value representing the whole channel. We can do that by **weighing** each value with the probability of each column happening, that is by $p_Y(y)$. In our case the second column ($W$ outputs $B$) happens with probability of $p_Y(B) = \frac{1}{2}$ and the third column ($W$ outputs $C$) happens with probability of $p_Y(C) = \frac{1}{2}$. So if we combine them we get 

$$
\begin{align}
V(\pi, C) &= \frac{1}{2} \cdot \frac{2}{3} + \frac{1}{2} \cdot \frac{2}{3} \\
V(\pi, C) &= \frac{2}{3}
\end{align}
$$

which is what we intuitively got without knowing the general definition of posterior vulnerability.

## libqif
We can also arrive to the same conclusions using python and libqif.

We can get $W$ posterior distributions using

In [11]:
P = channel.posteriors(W, pi)
print(P)

[[       nan 0.33333333 0.33333333]
 [       nan 0.         0.66666667]
 [       nan 0.66666667 0.        ]]


or get the channel's prior or posterior vulnerability with

In [17]:
prior_vuln = measure.bayes_vuln.prior(pi)
posterior_vuln = measure.bayes_vuln.posterior(pi, W)
print("Prior Bayes vulnerability:", prior_vuln)
print("Posterior Bayes vulnerability =", posterior_vuln)

Prior Bayes vulnerability: 0.3333333333333333
Posterior Bayes vulnerability = 0.6666666666666666


or the best guessing strategy for our scenario

In [14]:
print("Best guessing strategy =", measure.bayes_vuln.strategy(pi, W))

Best guessing strategy = [0 2 1]


Here 0 corresponds to Alice, 1 to bob and 2 to Charlie. The first element is for when $W$'s output is $A$, the second for when $W$'s output is $B$ and the third for $C$. The first element in our case does not have a meaning, because remember $W$ never outputs $A$. 

Compare libqif's results with the ones we computed ourselves. Did we get everything right?